## Overview

## Business Problem

## Data Understanding
For this analysis, we will utilize the "king County Housing Price from May 2014- May 2015" created by the Center for Spatial Data Science. It contains


In [ ]:
#import necessary packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats
from sklearn.feature_selection import RFE
from scipy.stats import kurtosis, skew
from sklearn.dummy import DummyRegressor

%matplotlib inline

In [ ]:
#ignore pairplot and graph warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
housing = pd.read_csv('data/kc_house_data.csv')
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing.isnull().sum()

Observations: We have several columns with categorical values
- waterfront      
- view             
- condition       
- grade          
- floors 
- zipcode

Also the column sqft_basement is an object. Let's find out why.

We have several null values in the waterfont and yr_renovated categories. we have a small number of null values in the view category. Yr_renovated seems self-explanatory - a nan value means that the
house was not renovated. Let's explore the remaining two categories. 


## Data Preparation 

- figuring out how to replace null values in columns
- investigating duplicate id
- creating null columns

In [ ]:
#housing['date'] =  pd.to_datetime(housing['date'])
#housing

In [ ]:
housing['id'].value_counts()

In [ ]:
idcheck1 = housing[housing['id'] == 795000620]
idcheck1

In [ ]:
idcheck2 = housing[housing['id'] == 1825069031]
idcheck2

In [ ]:
print(housing['waterfront'].isna().sum())
housing['waterfront'].value_counts()

In [ ]:
housing[(housing['waterfront'].isna()) & (housing['view'] == "NONE")]


In [ ]:
housing['waterfront'].fillna(value='NO', inplace=True)

**Waterfront view**

- the only options are yes or no. 
Any property on a waterfront will have a value other than NONE in the view column. Since there are 2110 rows that have a NaN value in waterfront and a none value in the view column in our data, it seems safer to assume NO as the default for waterfront.

We can either assume a property with a nan value for waterfront means it is not on the waterfront and replace all the nan values with no, or we can just drop the rows with a nan response. 
Since our data comprises of ~21,600 different rows, if we dropped these 2376 rows, we'd be loosing 
more than 10% of our data. 

So let's keep them and assume a nan response means no waterfront view. 

In [ ]:
#454 columns with ? for a value. 
housing['sqft_basement'].value_counts()

In [ ]:
# create new column sqrt_basement2 which is sqrt_living - sqrt-above. Addresses sqft_basement ? values.
housing['sqft_basement2'] = housing['sqft_living'] - housing['sqft_above']
housing.head()

In [ ]:
# has 3842 nan values, also has some 0.0 values
print(housing['yr_renovated'].isna().sum())
housing['yr_renovated'].value_counts()

In [ ]:
#creating new column was_renovated. Assuming Nan value and 0.0 - means home was not renovated.
housing['yr_renovated'].fillna(0, inplace=True)
housing['was_renovated'] = housing['yr_renovated'] != 0.0

In [ ]:
#replacing 0.0 in yr_renovated column with associated value in yr_built
housing.loc[housing['yr_renovated'] == 0, ['yr_renovated']] = housing['yr_built']

In [ ]:
#checking dataframe
housing.head()

In [ ]:
print(housing['view'].isna().sum())
housing['view'].value_counts()

In [ ]:
housing['view'].fillna(value='NONE', inplace=True)

**View**

- several options - none, nan, good, excellent, average, and fair.

We can either assume a nan response means no view and replace all the nan values with none, 
or we can just drop the rows with a nan response.  
Since our data comprises of ~21,600 different rows, if we dropped these 63 rows, 
we'd only be loosing 0.3% of our data.

Let's keep them and assume a nan response means no view.

**Sqft_basement**
- created new column to address the '?' values and the 0.0 values.

**Yr_renovated**
- created new column that addresses the 0.0 values and the nan values.

In [ ]:
#investigating outlier bedrooms
housing[housing['bedrooms'] > 8]

In [ ]:
#replacing 33 bedrooms with 3. we feel confident in assuming this was a typo based on the bathrooms and sqft_living
housing['bedrooms'] = housing['bedrooms'].replace(33, 3)

In [ ]:
housing.head()

In [ ]:
##exporting cleaned data frame
housing.to_csv('./data/kc_house_data_cleaned.csv')

## Feature Analysis
Let's further explore some of our variables.

In [ ]:
![housing_location](./images/housing_location.png)

In [ ]:
![housing_location](./images/housing_location.png)

In [ ]:
##using QGIS import create distance from waterbody column.
cleaned_housing = pd.read_csv('data/kc_water_dist_homes.csv')
cleaned_housing.head()

In [ ]:
min = cleaned_housing['price'].min()
max = cleaned_housing['price'].max()
mean = cleaned_housing['price'].mean()

print (f"The sale price range of homes sold is {min} to {max}")
print (f"The mean sale price of homes was {mean}")

In [ ]:
sns.distplot(cleaned_housing['price']);

In [ ]:
label = cleaned_housing['price']
fig, ax = plt.subplots(2, 1, figsize = (9,12))

# Plot the histogram   
ax[0].hist(label, bins=100)
ax[0].set_ylabel('Frequency')
ax[0].axvline(label.mean(), color='magenta', linestyle='dashed', linewidth=4)
ax[0].axvline(label.median(), color='cyan', linestyle='dashed', linewidth=4)

# Plot the boxplot   
ax[1].boxplot(label, vert=False)
ax[1].set_xlabel('price')
fig.suptitle('Price Distribution');

In [ ]:
print ('Skewness =', stats.skew(cleaned_housing['price']))
print ('Kurtosis =', stats.kurtosis(cleaned_housing['price']))

- price is normally distributed but has a significant right tail skew.
- Since the peak of the distribution is to the left of our mean, price is positively skewed.    
- This means that more than half of the houses in our dataset sold for less than the average price $540,000. 
- Our kurtosis and skew are high - we expect to see a positive skew and tail.
- Looking at our box plot - this illustrates that clearly - we have a number of outliers that sold for 
  significantly more than our average.
- Moving forward, let's remove some of our highest priced homes. Let's focus on homes that sold for less than           1,500,000.

In [ ]:
cleaned_housing.drop(cleaned_housing[ cleaned_housing['price'] >= 1500000 ].index, inplace = True)

In [ ]:
#sns.distplot(cleaned_housing['price']);

label = cleaned_housing['price']
fig, ax = plt.subplots(2, 1, figsize = (9,12))

# Plot the histogram   
ax[0].hist(label, bins=100)
ax[0].set_ylabel('Frequency')
ax[0].axvline(label.mean(), color='magenta', linestyle='dashed', linewidth=4)
ax[0].axvline(label.median(), color='cyan', linestyle='dashed', linewidth=4)

# Plot the boxplot   
ax[1].boxplot(label, vert=False)
ax[1].set_xlabel('price')
fig.suptitle('Price Distribution');

In [ ]:
## creating new column ratios to adjust for multicollinearity between two independent variables
cleaned_housing['bed_bath_ratio'] = (cleaned_housing['bedrooms'] /  cleaned_housing['bathrooms'])
cleaned_housing['sqft_living_to_bedroom_ratio'] = (cleaned_housing['sqft_living'] /  cleaned_housing['bedrooms'])
cleaned_housing['sqft_living_to_bathroom_ratio'] = (cleaned_housing['sqft_living'] /  cleaned_housing['bathrooms'])
cleaned_housing['ratio_sqft_lot_living'] = (cleaned_housing['sqft_lot'] /  cleaned_housing['sqft_living'])
cleaned_housing['ratio_sqft_living_lot'] = (cleaned_housing['sqft_living'] /  cleaned_housing['sqft_lot'])

In [ ]:
cleaned_housing['date'] =  pd.to_datetime(cleaned_housing['date'])

#cleaned_housing['date'] =  cleaned_housing['date'].astype(str)

#cleaned_housing['date'] = cleaned_housing['date'].dt.strftime('%d/%m/%Y')
#cleaned_housing['date'] = pd.to_datetime(cleaned_housing['date'], format='%Y/%m/%d')
#cleaned_housing['date'] = pd.to_datetime(cleaned_housing['date'], format='%m/%d/%Y')


In [ ]:
#cleaned_housing['age_at_sale'] = (cleaned_housing['date'].str[4:].astype(int) - cleaned_housing['yr_built']).astype(int)

#cleaned_housing['age_renovation_at_sale'] = (cleaned_housing['date'].str[4:].astype(int) - cleaned_housing['yr_renovated']).astype(int)

In [ ]:
cleaned_housing.rename(columns = {'Hub distance_HubDist':'Distance_to_Water'}, inplace = True)

In [ ]:
cleaned_housing = cleaned_housing.drop(['field_1'], axis=1)
cleaned_housing = cleaned_housing.drop(['sqft_basement'], axis=1)
cleaned_housing['zipcode'] = cleaned_housing['zipcode'].astype(str)


In [ ]:
cleaned_housing.info()

In [ ]:
cleaned_housing['condition'].value_counts()

## Train Test Split

In [ ]:
#Setting up train test split
X = cleaned_housing.drop('price', axis=1)
y = cleaned_housing['price']

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=57)

In [ ]:
#Combining X_train and y_train to get train_df
train_df = pd.concat([y_train, X_train], axis=1)
train_df.head()

## Simple Regression

In [ ]:
cleaned_housing.corr().price.sort_values(ascending=False)

In [ ]:
mask = np.triu(np.ones_like(cleaned_housing.corr(), dtype=bool))

plt.figure(figsize=(20,10))

cor = cleaned_housing.corr().abs()
sns.heatmap(cor, mask=mask, annot=True);

In [ ]:
#Model 1 - Simple Regression 1
simple_formula = 'price ~ sqft_living'
simple_model = sm.formula.ols(formula=simple_formula, data=train_df)
simple_model.summary = simple_model.fit().summary()

simple_model.summary

In [ ]:
sns.histplot(simple_model.resid);

In [ ]:
simple_model = simple_model.resid

fig, ax = plt.subplots(1,2,figsize=(15, 5))
ax[0].scatter(x=simple_model.fittedvalues,y=resid_simple_model)
ax[0].set_xlabel("Predicted Values")
ax[0].set_ylabel("Residual Error")
ax[0].set_title(label="Test for Homoscedasticity")


ax[1].hist(simple_model)
ax[1].set_xlabel("Residual Error")
ax[1].set_ylabel("Count")
ax[1].set_title(label="Histogram of Residual Error");

import statsmodels.api as sm
plt.style.use('ggplot')
fig = sm.graphics.qqplot(simple_model, dist=stats.norm, line='45', fit=True)

In [ ]:
y_max = y.max()
y_min = y.min()

ax = sns.scatterplot(X=simple_model.fittedvalues, y=y)
ax.set(ylim=(y_min, y_max))
ax.set(xlim=(y_min, y_max))
ax.set_xlabel("Predicted Sale Price")
ax.set_ylabel("Actual Sale Price")

X_ref = y_ref = np.linspace(y_min, y_max, 100)
plt.plot(X_ref, y_ref, color='red', linewidth=1)
plt.show()

In [ ]:
#simple_model_1_train_preds = simple_model.predict(sm.add_constant(X_train['sqft_living']))
#simple_model_1_train_preds
simple_train_preds = simple_model.predict(X_train['sqft_living'])

In [ ]:
#Plot our points, rating vs balance, as a scatterplot
plt.scatter(train_df['sqft_living'], train_df['price'])

# Plot the line of best fit!
plt.plot(train_df['sqft_living'], simple_train_preds, color='black')

plt.ylabel('Home Sale Price')
plt.xlabel('Sqft_living')
plt.title('Relationship between Home Sale Price and Sqft living space')
plt.show()

In [ ]:
# One last thing - can visualize both train and test set!

# Plot our training data
plt.scatter(train_df['Rating'], train_df['Balance'], color='blue', label='Training')
# Plot our testing data
plt.scatter(test_df['Rating'], test_df['Balance'], color='green', label='Testing')


# Plot the line of best fit
plt.plot(train_df['Rating'], simple_train_preds, color='black')
# Plotting for the test data just to show it's the same!
plt.plot(test_df['Rating'], simple_test_preds, color='red')

plt.ylabel('Credit Card Balance')
plt.xlabel('Credit Rating')
plt.title('Relationship between Credit Rating and Credit Card Balance')
plt.legend()
plt.show()

In [ ]:
visualizer = ResidualsPlot(model, hist=False, qqplot=True)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()

### Observations

- y = 198.74 * ('sqft') - 99,610. 
- Sqft_living accounts for about 43.6% of the variance in our sale price
- Each unit increase of Sqft_living increases the selling price of homes by on average about $200.


In [ ]:
#Model 2 - Simple Regression 2
simple_formula_2 = 'price ~ sqft_living15'
simple_model_2 = sm.formula.ols(formula=simple_formula_2, data=train_df)
simple_model_2summary = simple_model_2.fit().summary()

simple_model_2summary

In [ ]:
##input assumption visuals ## y_scaled= np.log(y)??

### Observations

- y = 225.16 * ('sqft') - 60,800. 
- the square footage living space for the nearest 15 homes account for 35% of the variance in our sale price
- Each unit increase of Sqft_living15 increases the selling price of homes on average by about $225.


In [ ]:
cat_cols = [c for c in train_df.columns if train_df[c].dtype == 'O']
cat_cols

In [ ]:
train_df.columns

In [ ]:
# create an encoder object. This will help us to convert categorical variables to new columns
encoder = OneHotEncoder(handle_unknown='error',
                        drop='first', 
                        categories='auto')

ct = ColumnTransformer(transformers=[('ohe', encoder, cat_cols)],
                       remainder='passthrough')
ct.fit(X_train) 
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

In [ ]:
#create dummy variables for the "condition" column
condition_dummies = pd.get_dummies(X_train_condition['condition'], drop_first=True)
condition_dummies
#drops 'Average', creates 4 additional columns

In [ ]:
X_train_dummies = pd.concat([X_train_condition, condition_dummies], axis=1)
X_train_dummies

In [ ]:
#Model 3 - Multiple Regression 1
Multiple_formula = 'price ~ sqft_living + yr_built + Distance_to_Water + bed_bath_ratio'
Multiple_model = sm.formula.ols(formula=Multiple_formula, data=train_df)
Multiple_model_summary = Multiple_model.fit().summary()

Multiple_model_summary

In [ ]:
#Model 3 - Multiple Regression 2
#add in condition and zip code
Multiple_formula_2 = 'price ~ sqft_living + yr_built + Distance_to_Water + bed_bath_ratio'
Multiple_model_2 = sm.formula.ols(formula=Multiple_formula_2, data=train_df)
Multiple_model_2summary = Multiple_model_2.fit().summary()

Multiple_model_2summary

CODE

In [ ]:
sns.pairplot(housing)
plt.show()

In [ ]:
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select =4)

In [ ]:
ss = StandardScaler()
ss.fit(cleaned_housing.drop('price', axis=1))
cleaned_housing_scaled = ss.transform(cleaned_housing.drop('price', axis=1))

In [ ]:
cleaned_housing_scaled

In [ ]:
select.fit(X=cleaned_housing_scaled, y=cleaned_housing['price'])

In [ ]:
select.support_

In [ ]:
select.ranking_

In [ ]:
## use sqft_living  yr_built  sqft_living15  sqft_lot15

In [ ]:
Polynomials using all except categorical values

In [ ]:
X = cleaned_housing.drop('price', axis=1)
y = cleaned_housing['price']
pf = PolynomialFeatures(degree=3)
pf.fit(X)

In [ ]:
pf.transform(X)

In [ ]:
pf.transform(X).shape

In [ ]:
pf.get_feature_names()

In [ ]:
polynomial_df = pd.DataFrame(pf.transform(X), columns= pf.get_feature_names() )

In [ ]:
lr = LinearRegression()
lr.fit(polynomial_df, y)

In [ ]:
lr.score(polynomial_df, y)

## DO NOT RUN CELL BELOW 